In [ ]:
pip install kaggle

In [37]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [38]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
^C


In [39]:
from zipfile import ZipFile

dataset_path = '/Users/lejladoric/Desktop/twitter-sentiment-analysis/venv/sentiment140.zip'
with ZipFile(dataset_path, 'r') as zip:
    zip.extractall()
    print("The dataset is extracted.")

The dataset is extracted.


In [60]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lejladoric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [42]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [43]:
twitter_data= pd.read_csv("/Users/lejladoric/Desktop/twitter-sentiment-analysis/venv/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1")



In [44]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

twitter_data = pd.read_csv("/Users/lejladoric/Desktop/twitter-sentiment-analysis/venv/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", names=column_names)


In [45]:
twitter_data.shape

(1600000, 6)

In [46]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [47]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [48]:
twitter_data.replace({'target': {4:1}}, inplace=True)   

In [49]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

Stemming - reducing words to the root word

In [50]:
port_stemmer = PorterStemmer()



In [ ]:
def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stemmer.stem(word) for word in stemmed_content if not word in set(stopwords.words('english'))]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content


In [52]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [53]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [54]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [55]:
X = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [56]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train) # we use this on the training data to learn the vocabulary 
X_test = vectorizer.transform(X_test) # we use this on the test data to transform it using the vocabulary learned from the training data but not to learn a new vocabulary by fitting it again 



In [58]:
print(X_train)
print(y_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

Training the model

In [61]:
model  = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


Model Evaluation


In [65]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.79871953125


In [67]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test, X_test_prediction)
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.77668125


Saving the model

In [68]:
import pickle

filename= 'logistic_regression_model.sav'
pickle.dump(model, open(filename, 'wb'))
print("Model saved successfully.")

Model saved successfully.


Using the saved model for future predictions

In [70]:
loaded_model = pickle.load(open(filename, 'rb'))




In [78]:
X_new = X_test[6]
prediction = loaded_model.predict(X_new)
print(prediction)
print("The original correct output is: ", y_test[6])

if prediction[0] == 1:
    print("The sentiment is positive.")
else:
    print("The sentiment is negative.")

[1]
The original correct output is:  1
The sentiment is positive.
